In [1]:
%run utils.py


class SimpleNeuralNet(nn.Module): 

    def __init__(self, num_inputs, num_hidden, num_outputs):
        super().__init__()
        # Initialize the modules we need to build the network
        self.linear1 = nn.Linear(num_inputs, num_hidden)
        self.act_fn = nn.ReLU()
        self.linear2 = nn.Linear(num_hidden, num_outputs)

    def forward(self, x):
        # Perform the calculation of the model to determine the prediction
        x = self.linear1(x)
        x = self.act_fn(x)
        x = self.linear2(x)
        return x
    
    
class Dataset():

    def __init__(self, size, input_size, output_size, start_from=0):
     
        super().__init__()
        self.entries = size
        self.X = torch.zeros([size, input_size], dtype=torch.float32)
        self.Y = torch.zeros([size, output_size], dtype=torch.float32)
        
        #get the datapoints one by one and build the dataset : To be improved computationnaly !!
        for i in tqdm(range(size)) :
            file_name = "Output/5by5_Data{}".format(start_from+i)
            x,y = create_datapoint(file_name, 5, 5, 3 )
            self.X[i] = x
            self.Y[i] = y

    def __len__(self):
        return self.entries

    def __getitem__(self, idx):
        data_point = self.X[idx]
        data_label = self.Y[idx]
        return data_point, data_label

In [2]:
train_dataset_size = 30000
train_dataset = Dataset(train_dataset_size, 3025, 1155 )  # 3025 is the X size ; 1155 is the Y size
train_data_loader = data.DataLoader(train_dataset, batch_size=128, shuffle=False, drop_last=False)
    
model = SimpleNeuralNet(num_inputs=3025, num_hidden=16, num_outputs=1155)
#for name, param in model.named_parameters():
    #print(f"Parameter {name}, shape {param.shape}")  # weight [output, input],  bias [output] 
    
#print(model)

  0%|          | 0/30000 [00:00<?, ?it/s]

In [3]:
loss_module = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)
#optimizer = torch.optim.AdamW(model.parameters(),weight_decay=0.375e-2) 

def train_model(model, optimizer, data_loader, loss_module, num_epochs=1000):
    losses = []
    # Set model to train mode
    model.train()

    # Training loop
    for epoch in tqdm(range(num_epochs)):
        for data_inputs, data_labels in data_loader:
            preds = model(data_inputs)
            assert preds.shape == data_labels.shape 
            loss = loss_module(preds, data_labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
    return losses

losses = train_model(model, optimizer, train_data_loader, loss_module, 1000)

  0%|          | 0/1000 [00:00<?, ?it/s]

In [16]:
test_dataset_size = 10000
test_dataset = Dataset(test_dataset_size, 3025, 1155, start_from=30000 ) 
test_data_loader = data.DataLoader(test_dataset, batch_size=128, shuffle=False, drop_last=False)

def eval_model(model, data_loader):
    
    predictions = []
    
    model.eval() # Set model to eval mode
    with torch.no_grad(): # Deactivate gradients for the following code
        for data_inputs, data_labels in data_loader:
            preds = model(data_inputs)
            predictions.append(preds)
    return predictions
P = eval_model(model, test_data_loader)
predicted = torch.stack(flatten_chain(P), dim=0)

print("test score", mean_squared_error(test_dataset.Y, predicted))

  0%|          | 0/10000 [00:00<?, ?it/s]

test score 141914.83
